# TensorQTL QTL association testing

This notebook implements a workflow using [tensorQTL](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-019-1836-7) to perform QTL association testing.

## Input

- List of molecular phenotype files: a list of `bed.gz` files containing the table for the molecular phenotype. It should have a companion index file in `tbi` format.

    The header of the bed.gz is per the [TensorQTL](https://github.com/broadinstitute/tensorqtl) convention:

    >    Phenotypes must be provided in BED format, with a single header line starting with # and the first four columns corresponding to: chr, start, end, phenotype_id, with the remaining columns corresponding to samples (the identifiers must match those in the genotype input). The BED file should specify the center of the cis-window (usually the TSS), with start == end-1.


- List of genotypes in PLINK binary format (`bed`/`bim`/`fam`) for each chromosome, previously processed through our genotype QC pipelines.
- Covariate file, a file with #id + samples name as colnames and each row a covariate: fixed and known covariates as well as hidden covariates recovered from factor analysis.
- Optionally, a list of traits (genes, regions of molecular features etc) to analyze.

For cis-analysis:

- Optionally, a list of genomic regions associate with each molecular features to analyze. The default cis-analysis will use a window around TSS. This can be customized to take given start and end genomic coordinates.


## Output

For each chromosome, several of summary statistics files are generated, including both nominal test statistics for each test, as well as region (gene) level association evidence.

The columns of nominal association result are as follows:

- phenotype_id: Molecular trait identifier.(gene)
- variant_id: ID of the variant (rsid or chr:position:ref:alt)
- tss_distance: Distance of the SNP to the gene transcription start site (TSS)
- af: The allele frequency of this SNPs
- ma_samples: Number of samples carrying the minor allele
- ma_count: Total number of minor alleles across individuals
- pval: Nominal P-value from linear regression
- beta: Slope of the linear regression
- se: Standard error of beta
- chr : Variant chromosome.
- pos : Variant chromosomal position (basepairs).
- ref : Variant reference allele (A, C, T, or G).
- alt : Variant alternate allele.


The column specification of region (gene) level association evidence are as follows:

- phenotype_id - Molecular trait identifier. (gene)
- num_var - Total number of variants tested in cis
- beta_shape1 - First parameter value of the fitted beta distribution
- beta_shape2 - Second parameter value of the fitted beta distribution
- true_df - Effective degrees of freedom the beta distribution approximation
- pval_true_df - Empirical P-value for the beta distribution approximation
- variant_id - ID of the top variant (rsid or chr:position:ref:alt)
- tss_distance - Distance of the SNP to the gene transcription start site (TSS)
- ma_samples - Number of samples carrying the minor allele
- ma_count - Total number of minor alleles across individuals
- maf - Minor allele frequency in MiGA cohort
- ref_factor - Flag indicating if the alternative allele is the minor allele in the cohort (1 if AF <= 0.5, -1 if not)
- pval_nominal - Nominal P-value from linear regression
- slope - Slope of the linear regression
- slope_se - Standard error of the slope
- pval_perm - First permutation P-value directly obtained from the permutations with the direct method
- pval_beta - Second permutation P-value obtained via beta approximation. This is the one to use for downstream analysis

# Command interface 

In [5]:
sos run TensorQTL.ipynb -h

usage: sos run TensorQTL.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  cis
  trans

Global Workflow Options:
  --covariate-file VAL (as path, required)
                        Covariate file
  --genotype-file VAL (as path, required)
                        For cis, genotype file in PLINK binary format
                        (bed/bam/fam) format, per chrom, for trans, 1 whole
                        genome genotype file in plink binary format
  --region-list . (as path)
                        An optional subset of regions of molecular features to
                        analyze
  --customized-cis-windows . (as path)
                        An optional list documenting the cus

## Minimal working example

An MWE is uploaded to [google drive](https://drive.google.com/drive/folders/1yjTwoO0DYGi-J9ouMsh9fHKfDmsXJ_4I?usp=sharing).
The singularity image (sif) for running this MWE is uploaded to [google drive](https://drive.google.com/drive/folders/1mLOS3AVQM8yTaWtCbO8Q3xla98Nr5bZQ)

FIXME: need to update these links. 

FIXME: Also need to update the example commands below using our new example dataset.

In [ ]:
sos run pipeline/TensorQTL.ipynb cis \
    --genotype-file plink_files_list.txt \
    --phenotype-file MWE.bed.recipe \
    --covariate-file ALL.covariate.pca.BiCV.cov.gz \
    --cwd ./output/ \
    --container containers/TensorQTL.sif --MAC 5

In [ ]:
sos run pipeline/TensorQTL.ipynb trans \
    --genotype-file MWE.bed \
    --phenotype-file MWE.log2cpm.mol_phe.bed.gz \
    --covariate-file ALL.covariate.pca.BiCV.cov.gz \
    --cwd ./output/ \
    --container containers/TensorQTL.sif --MAC 5 --region-name  gene_name

In [ ]:
sos run pipeline/TensorQTL.ipynb trans \
    --genotype-file /mnt/vast/hpc/csg/snuc_pseudo_bulk/data/genotype_qced/GRCh38_liftedover_sorted_all.add_chr.leftnorm.filtered.renamed.filtered.renamed.filtered.filtered.bed \
    --phenotype-file /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_tissue_analysis/output/data_preprocessing/ALL/phenotype_data/ALL.log2cpm.bed.gz \
    --covariate-file /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_tissue_analysis/output/data_preprocessing/ALL/covariates/ALL.log2cpm.ALL.covariate.pca.resid.PEER.cov.gz \
    --cwd ./output/trans_tensorQTL/ \
    --region-list /mnt/vast/hpc/csg/snuc_pseudo_bulk/eight_tissue_analysis/reference_data/AD_genes.region_list \
    --container containers/TensorQTL.sif --MAC 5 --region-name ID

## Global parameter settings

In [5]:
[global]
# Covariate file
parameter: covariate_file = path
# For cis, genotype file in PLINK binary format (bed/bam/fam) format, per chrom, for trans, 1 whole genome genotype file in plink binary format
parameter: genotype_file = path
# An optional subset of regions of molecular features to analyze. The last column is the  gene names
parameter: region_list = path()
# An optional list documenting the custom cis window for each region to analyze, with four column, chr, start, end, region ID (eg gene ID).
# If this list is not provided, the default `window` parameter (see below) will be used.
parameter: customized_cis_windows = path()
# Path to the work directory of the analysis.
parameter: cwd = path('output')
# Phenotype file, if cis a list of phenotype per chrom, if trans, 1 whole genome phenotype file.
parameter: phenotype_file = path
# Prefix for the analysis output
parameter: name = f"{phenotype_file:bn}_{covariate_file:bn}"
# Minor allele count cutoff
parameter: MAC = 0
# The name of phenotype corresponding to gene_id or gene_name in the region
parameter: region_name = "gene_id"
# The phenotype group file to group molecule_trait into molecule_trait_object
# This is applicable to when there are multiple molecular events in the same region, such as sQTL analysis.
parameter: phenotype_group = path() 

# Specify the cis window for the up and downstream radius to analyze around the region of interest, in units of bp
# This parameter will be set to zero if `customized_cis_windows` is provided.
parameter: window = 1000000

# Number of threads
parameter: numThreads = 8
# For cluster jobs, number commands to run per job
parameter: job_size = 1
parameter: walltime = '12h'
parameter: mem = '16G'
# Container option for software to run the analysis: docker or singularity
parameter: container = ''
parameter: entrypoint={('micromamba run -a "" -n' + ' ' + container.split('/')[-1][:-4]) if container.endswith('.sif') else f''}

# Use the header of covariate file to decide the sample size
import pandas as pd
N = len(pd.read_csv(covariate_file, sep = "\t",nrows = 1).columns) - 1

# Minor allele frequency cutoff. It will overwrite minor allele cutoff.
parameter: maf_threshold = MAC/(2.0*N)

## cis-xQTL association testing

In [ ]:
[cis_1]
# parse input file lists
import pandas as pd
molecular_pheno_files = pd.read_csv(phenotype_file, sep = "\t")
genotype_files = pd.read_csv(genotype_file,sep = "\t")
genotype_files["#id"] = genotype_files["#id"].astype(str)
molecular_pheno_files["#id"] = molecular_pheno_files["#id"].astype(str)

input_files = molecular_pheno_files.merge(genotype_files, on = "#id")
input_files = input_files.values.tolist()
input_chroms = [x[0] for x in input_files]
input_files = [x[1:] for x in input_files]

# skip nominal association results if the files exists already
# This is false by default which means to recompute everything
# This is only relevant when the `parquet` files for nominal results exist but not the other files
# and you want to avoid computing the nominal results again
parameter: skip_nominal_if_exist = False
input: input_files, group_by = len(input_files[0]), group_with = "input_chroms" 
output: parquet = f'{cwd:a}/{name}.cis_qtl_pairs.{_input_chroms}.parquet', # This convention is necessary to match the pattern of map_norminal output
        regional = f'{cwd:a}/{name}.{_input_chroms}.cis_qtl_regional.gz',
        nominal = f'{cwd:a}/{name}.{_input_chroms}.cis_qtl_nominal.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
python: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout' , container = container, entrypoint = entrypoint
    import pandas as pd
    import numpy as np
    import tensorqtl
    from tensorqtl import genotypeio, cis, trans
    from scipy.stats import chi2
        
    ## Define paths
    plink_prefix_path = $[_input[1]:nar]
    expression_bed = $[_input[0]:ar]
    covariates_file = "$[covariate_file:a]"
    window = $[window]
    ## Load Data
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
    phenotype_id = phenotype_pos_df.index.name
    ## Analyze only the regions listed
    if $[region_list.is_file()]:
        region = pd.read_csv("$[region_list:a]", comment="#", header=None, sep="\t")
        keep_region = region.iloc[:, -1].to_list()
        phenotype_df = phenotype_df[phenotype_df.index.isin(keep_region)]
        phenotype_pos_df = phenotype_pos_df[phenotype_pos_df.index.isin(keep_region)]
    ## use custom cis windows
    if $[customized_cis_windows.is_file()]:
        cis_list = pd.read_csv("$[customized_cis_windows:a]", comment="#", header=None, names=["chr","start","end",phenotype_id], sep="\t")
        phenotype_pos_df = phenotype_pos_df.reset_index() #move the phenotype id index to a new column of the dataframe
        phenotype_pos_df = phenotype_pos_df.merge(cis_list, left_on = ["chr",phenotype_id],right_on = [cis_list.columns[0],cis_list.columns[3]])#in some cases (gene expression for eQTLs) the phenotype_id may be in the cis_list file
        phenotype_pos_df = phenotype_pos_df.set_index(phenotype_id)[["chr","start","end"]] # The final phenotype_pos_df will have three columns(chr, start, end) and index is the phenotype ID
        if len(phenotype_df.index) != len(phenotype_pos_df.index):
            raise ValueError("cannot uniquely match all the phentoype data in the input to the customized cis windows provided")
        window = 0 # In the updated tensorQTL, by default if there is a customized cis window, the actual cis window will be start - window & end + window, so it is necessary to change the window parameter to 0
    
    covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T
    pr = genotypeio.PlinkReader(plink_prefix_path)
    genotype_df = pr.load_genotypes()
    variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]
    ## Retaining only common samples
    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, covariates_df.index)]
    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, genotype_df.columns)]
    covariates_df = covariates_df.transpose()[np.intersect1d(phenotype_df.columns, covariates_df.index)].transpose()
    
    ## To simplify things, there should really not be "chr" prefix
    phenotype_pos_df.chr = [x.replace("chr","") for x in phenotype_pos_df.chr]
    variant_df.chrom = [x.replace("chr","") for x in variant_df.chrom]
    ## Read phenotype group if availble
    if $[phenotype_group.is_file()]:
        group_s = pd.read_csv($[phenotype_group:r], sep='\t', header=None, index_col=0, squeeze=True)
    else:
        group_s = None

    ## cis-QTL mapping: nominal associations for all variant-phenotype pairs
    if not ($[skip_nominal_if_exist] and $[_output[0].is_file()]):
        cis.map_nominal(genotype_df, variant_df,
                phenotype_df,
                phenotype_pos_df,
                "$[_output[0]:nnn]", 
                covariates_df=covariates_df, 
                window=window, 
                maf_threshold = $[maf_threshold],
                group_s=group_s)

    ## Load the parquet and save it as txt
    pairs_df = pd.read_parquet("$[_output[0]]")
    ## Adds the group columns to pairs_df, if there is group_s use group_s, else use phenotype_id
    if group_s is not None:
        pairs_df = pairs_df.merge(pd.DataFrame( {"molecular_trait_object_id": group_s}),left_on = "phenotype_id", right_index = True)
    else:
        pairs_df["molecular_trait_object_id"] = pairs_df.phenotype_id

    # genomic inflation factor lambda
    lambda_col = pairs_df.groupby("molecular_trait_object_id").apply( lambda x:  chi2.ppf(1. - np.median(x.pval_nominal), 1)/chi2.ppf(0.5,1))
    pairs_df.columns.values[0]  = "molecular_trait_id"
    pairs_df.columns.values[6]  = "pvalue"
    pairs_df.columns.values[7]  = "beta"
    pairs_df.columns.values[8]  = "se"
    pairs_df = pairs_df.assign(maf = lambda dataframe: dataframe['af'].map(lambda af:af if af < 0.5 else 1-af) ).drop("af",axis =  1)
    pairs_df["n"] = len(phenotype_df.columns.values)
    pairs_df = pairs_df.assign(
    alt = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[-1])).assign(
    ref = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[-2])).assign(
    pos = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[0].split(":")[1])).assign(
    chrom = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split(":")[0]))
    pairs_df.to_csv("$[_output[2]]", sep='\t',index = None, compression='gzip')
    cis_df = cis.map_cis(genotype_df, 
                        variant_df, 
                        phenotype_df,
                        phenotype_pos_df,
                        covariates_df=covariates_df, 
                        seed=999, 
                        window=$[window], 
                        maf_threshold = $[maf_threshold],
                        group_s=group_s)
    
    cis_df.index.name = "molecular_trait_id"
    ## Add groups columns for eQTL analysis
    if "group_id" not in cis_df.columns:
        cis_df["group_id"] = cis_df.index
        cis_df["group_size"] = 1
    cis_df = cis_df.rename({"group_id":"molecular_trait_object_id","group_size":"n_traits","num_var" : "n_variants","variant_id":"variant","pval_perm":"p_perm", "pval_beta":"p_beta" },axis = 1)
    cis_df = cis_df.assign(genomic_inflation = lambda dataframe : dataframe["molecular_trait_object_id"].map(lambda molecular_trait_object_id:lambda_col[molecular_trait_object_id]))
    cis_df.to_csv("$[_output[1]]", sep='\t', compression='gzip')

bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container, entrypoint = entrypoint
        for i in $[_output[0]] ; do 
        echo "output_info: $i "
        echo "This is the file containing the immediate output of TensorQTL's map_nominal function "
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        done
        for i in $[_output[1]] ; do 
        echo "output_info: $i " 
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        echo "output_rows:" `zcat $i | wc -l  | cut -f 1 -d " "` 
        echo "output_headerow:" `zcat $i | grep "##" | wc -l ` 
        echo "output_column:" `zcat $i | grep -V "##" | head -1 | wc -w `
        echo "output_preview:"
        zcat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6,7,8,9,10  ; done
        for i in $[_output[2]] ; do 
        echo "output_info: $i "
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        echo "output_rows:" `zcat $i | wc -l  | cut -f 1 -d " "` 
        echo "output_headerow:" `zcat $i | grep "##" | wc -l `
        echo "output_column:" `zcat $i | grep -V "##" | head -1 | wc -w `
        echo "output_preview:" 
        zcat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6,7,8,9,10 ; done

## TransQTL association testing

For transQTL analysis, it is suggested to provide the largest memory and CPU threads available on a compute node. eg 250G and >32 threads.

In [1]:
[trans_1]
fail_if(not region_list.is_file(), msg = "For trans analysis, ``region_list`` must be provided.")
parameter: permutation_batch_size = 50000
parameter: pval_threshold = 1.0
parameter: permutation = False
input: phenotype_file, genotype_file 
output: nominal = f'{cwd:a}/{_input[0]:bnn}.trans_qtl_nominal.gz'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
python: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container =container, entrypoint = entrypoint
    import pandas as pd
    import numpy as np
    import tensorqtl
    from tensorqtl import genotypeio, cis, trans
    ## Define paths
    plink_prefix_path = $[_input[1]:nar]
    expression_bed = $[_input[0]:ar]
    covariates_file = "$[covariate_file:a]"
    window = $[window]
    ## Loading Data
    phenotype_df, phenotype_pos_df = tensorqtl.read_phenotype_bed(expression_bed)
    phenotype_id = phenotype_pos_df.index.name
    ## Analyze only the regions listed
    if $[region_list.is_file()]:
        region = pd.read_csv("$[region_list:a]", comment="#", header=None, sep="\t")
        keep_region = region.iloc[:, -1].to_list()
        phenotype_df = phenotype_df[phenotype_df.index.isin(keep_region)]
        phenotype_pos_df = phenotype_pos_df[phenotype_pos_df.index.isin(keep_region)]
    ## use custom cis windows
    if $[customized_cis_windows.is_file()]:
        cis_list = pd.read_csv("$[customized_cis_windows:a]", comment="#", header=None, names=["chr","start","end",phenotype_id], sep="\t")
        phenotype_pos_df = phenotype_pos_df.reset_index() #move the phenotype id index to a new column of the dataframe
        phenotype_pos_df = phenotype_pos_df.merge(cis_list, left_on = ["chr",phenotype_id],right_on = [cis_list.columns[0],cis_list.columns[3]])#in some cases (gene expression for eQTLs) the phenotype_id may be in the cis_list file
        phenotype_pos_df = phenotype_pos_df.set_index(phenotype_id)[["chr","start","end"]] # The final phenotype_pos_df will have three columns(chr, start, end) and index is the phenotype ID
        if len(phenotype_df.index) != len(phenotype_pos_df.index):
            raise ValueError("cannot uniquely match all the phentoype data in the input to the customized cis windows provided")
        window = 0 # In the updated tensorQTL, by default if there is a customized cis window, the actual cis window will be start - window & end + window, so it is necessary to change the window parameter to 0
    
    covariates_df = pd.read_csv(covariates_file, sep='\t', index_col=0).T
    pr = genotypeio.PlinkReader(plink_prefix_path)
    genotype_df = pr.load_genotypes()
    variant_df = pr.bim.set_index('snp')[['chrom', 'pos']]
    ## Retaining only common samples
    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, covariates_df.index)]
    phenotype_df = phenotype_df[np.intersect1d(phenotype_df.columns, genotype_df.columns)]
    covariates_df = covariates_df.transpose()[np.intersect1d(phenotype_df.columns, covariates_df.index)].transpose()
    ## Trans analysis
    trans_df = trans.map_trans(genotype_df, 
                            phenotype_df,
                            covariates_df, 
                            batch_size=$[permutation_batch_size],
                            return_sparse=True, 
                            return_r2 = True, 
                            pval_threshold=$[pval_threshold], 
                            maf_threshold=$[maf_threshold])
    ## Filter out cis signal, again if customized cis windows are used, the windows is [start-win,end + win] where win = 0, else it is [start - win, start + win]
    trans_df = trans.filter_cis(trans_df, phenotype_pos_df, variant_df, window=window)   
    ## Permutation
    if $["True" if permutation else "False"]:
        perm_df = trans.map_permutations(genotype_df, covariates_df, batch_size=$[permutation_batch_size],
                             maf_threshold=$[maf_threshold])
        perm_output = trans.apply_permutations(perm_df,trans_df)
    ## Output
    trans_df.columns.values[1]  = "molecular_trait_id"
    trans_df.columns.values[2]  = "pvalue"
    trans_df.columns.values[3]  = "beta"
    trans_df.columns.values[4]  = "se"
    trans_df = trans_df.assign(maf = lambda dataframe: dataframe['af'].map(lambda af:af if af < 0.5 else 1-af) ).drop("af",axis =  1)
    trans_df = trans_df.assign(
    chrom = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split(":")[0])).assign(
    alt = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[2])).assign(
    ref = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[1])).assign(
    pos = lambda dataframe: dataframe['variant_id'].map(lambda variant_id:variant_id.split("_")[0]))
    trans_df.to_csv("$[_output]", sep='\t',index = None)

bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container, entrypoint = entrypoint
        for i in $[_output] ; do 
        echo "output_info: $i "
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        echo "output_rows:" `cat $i | wc -l  | cut -f 1 -d " "`
        echo "output_headerow:" `cat $i | grep "##" | wc -l ` 
        echo "output_column:" `cat $i | grep -V "##" | head -1 | wc -w ` 
        echo "output_preview:" 
        cat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6,7,8,9,10; done

## Association results processing
For both cis and trans: Generate the recipe for yml processing
For cis: Also process the consolidates emprical data.

In [1]:
[cis_2]
input:  group_by = "all"
output: f'{_input["nominal"][0]:nnn}.cis_qtl.meta_info',
        f'{_input["nominal"][0]:nnn}.cis_qtl.column_info',
        f'{_input["regional"][0]:nnn}.fdr.gz',
        f'{_input["regional"][0]:nnn}.summary.txt'   
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container = container, entrypoint = entrypoint
    import pandas as pd

    data_temp = pd.DataFrame({
    "sumstat_dir" : [$[_input["regional"]:r,]],
    "column_info" : $[_output[1]:r]
    })
    column_info_df = pd.DataFrame( pd.Series( {"ID": "molecular_trait_id,chromosome,position,ref,alt",
      "chromosome": "chrom",
      "position": "pos",
      "variant": "variant_id",
      "ref": "ref",
      "alt": "alt",
      "beta": "beta",
      "se": "se",
      "pvalue": "pvalue",
      "TSS_D": "tss_distance",
      "maf": "maf",
      "n" : "n" ,
      "ma_samples": "ma_samples",
      "ac": "ma_count",
      "molecular_trait_id": "molecular_trait_id", 
      "molecular_trait_object_id": "molecular_trait_object_id"}), columns = ["TensorQTL"] )

    data_temp["#chr"] = [x.split(".")[-3].replace("chr","") for x in  [$[_input["nominal"]:r,]]]
    data_temp = data_temp[['#chr', 'sumstat_dir', 'column_info']]
    data_temp.to_csv("$[_output[0]]",index = False,sep = "\t" )
    column_info_df.to_csv("$[_output[1]]",index = True,sep = "\t" )

R: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container =container, entrypoint = entrypoint 
    library("purrr")
    library("tidyr")
    library("dplyr")
    library("readr")
    library("qvalue")
    emprical_pd = tibble(map(c($[_input["regional"]:r,]), ~read_delim(.x,"\t")))%>%unnest()
    emprical_pd["q_beta"] = tryCatch(qvalue(emprical_pd$p_beta)$qvalue, error = function(e){print("Too few pvalue to calculate qvalue, fall back to BH") 
                                                                                              qvalue(emprical_pd$p_beta,pi0 = 1 )$qvalue})  

    emprical_pd["q_perm"] = tryCatch(qvalue(emprical_pd$p_perm)$qvalue, error = function(e){print("Too few pvalue to calculate qvalue, fall back to BH") 
                                                                                              qvalue(emprical_pd$p_perm,pi0 = 1 )$qvalue})
    emprical_pd["fdr_beta"] = p.adjust(emprical_pd$p_beta,"fdr")    
    emprical_pd["fdr_perm"] = p.adjust(emprical_pd$p_perm,"fdr")    
    summary = tibble("fdr_perm_0.05" =  sum(emprical_pd["fdr_perm"] < 0.05) , 
                      "fdr_beta_0.05" = sum(emprical_pd["fdr_beta"] < 0.05),
                      "q_perm_0.05" = sum(emprical_pd["q_perm"] < 0.05) ,
                      "q_beta_0.05" = sum(emprical_pd["q_beta"] < 0.05) ,
                       "q_perm_0.01" = sum(emprical_pd["q_perm"] < 0.01) ,
                      "q_beta_0.01" = sum(emprical_pd["q_beta"] < 0.01)  )
    emprical_pd%>%write_delim("$[_output[2]]","\t")
    summary%>%write_delim("$[_output[3]]","\t")
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container, entrypoint = entrypoint
        for i in $[_output] ; do 
        echo "output_info: $i "
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        echo "output_rows:" `cat $i | wc -l  | cut -f 1 -d " "`
        echo "output_headerow:" `cat $i | grep "##" | wc -l `
        echo "output_column:" `cat $i | grep -V "##" | head -1 | wc -w `
        echo "output_preview:"
        cat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6,7,8,9,10 ; done

In [ ]:
[trans_2]
input:  group_by = "all"
output: f'{_input["nominal"]:nn}.trans_qtl.meta_info',
        f'{_input["nominal"]:nn}.trans_qtl.column_info'
python: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container, entrypoint = entrypoint
    import pandas as pd 
    data_temp = pd.DataFrame({
    "sumstat_dir" : [$[_input["nominal"]:r,]],
    "column_info" : $[_output[1]:r]
    })
    column_info_df = pd.DataFrame( pd.Series( {"ID": "molecular_trait_id,chromosome,position,ref,alt",
      "chromosome": "chrom",
      "position": "pos",
      "ref": "ref",
      "alt": "alt",
      "variant": "variant_id",
      "beta": "beta",
      "se": "se",
      "pvalue": "pval",
      "maf": "maf",
      "molecular_trait_id": "gene_ID"}), columns = ["TensorQTL"] )
    data_temp.to_csv("$[_output[0]]",index = False,sep = "\t" )
    column_info_df.to_csv("$[_output[1]]",index = True,sep = "\t" )

bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout', container = container, entrypoint = entrypoint
        for i in $[_output] ; do 
        echo "output_info: $i "
        echo "output_size:" `ls -lh $i | cut -f 5  -d  " "` 
        echo "output_rows:" `cat $i | wc -l  | cut -f 1 -d " "` 
        echo "output_headerow:" `cat $i | grep "##" | wc -l ` 
        echo "output_column:" `cat $i | grep -V "##" | head -1 | wc -w ` 
        echo "output_preview:" 
        cat $i  | grep -v "##" | head  | cut -f 1,2,3,4,5,6,7,8,9,10 ; done